In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV

import import_ipynb
# Импортируем полностью модули для передачи глобальных переменных
from utils import make_datasets
import models

# Построение гибридной системы

## Объединение моделей

In [20]:
def get_hybrid_recommendation(user_id, df_ratings, df_books, df_tags, df_book_tags, book_id=None, weights=None):
    """Функция объединяет прогнозы разных моделей и выводит список рекомендуемых книг.
    :param user_id: идентификатор пользователя
    :param book_id: идентификатор книги
    :param df_ratings: DataFrame с рейтингами
    :param df_books: DataFrame с информацией о книгах
    :param df_tags: DataFrame с тегами
    :param df_book_tags: DataFrame с привязанными тегами к книгам
    :param weights: весовые коэффициенты для моделей [популярность, похожесть, коллаборативная фильтрация]

    :return: список рекомендованных книг"""
    
    # Определим веса для каждой из рекомендаций
    if weights is None:
        weights = [2, 3, 4]
    
    # Получаем рекомендации книг по каждой модели в соответствии с весами
    popularity_rec = models.get_popularity_recommendation_ids(df_ratings, weights[0])
    content_rec = models.get_similar_books_ids(df_book_tags, df_tags, df_books, df_ratings, book_id, weights[1])
    collaborative_rec = models.get_recommendations_svd(user_id, df_ratings, weights[2])
    
    # Отбираем уникальные книги
    recommendations = (popularity_rec + content_rec + collaborative_rec)
    unique_recs = list(set(recommendations))
    
    return unique_recs

In [29]:
def get_user_type(user_id, df_users, threshold=None):
    """Функция классификацириует тип пользователя: новый или активный.
Основывается на количестве прочтённых книг и средней оценке.
    :param user_id: идентификатор пользователя
    :param df_users: DataFrame c информацией о пользователях

    :return: строка ('new' или 'active')
    """
    # Находим характеристики пользователя
    avg_user_rating = df_users[df_users['user_id'] == user_id]['avg_user_rating'].iloc[0]
    num_user_ratings = df_users[df_users['user_id'] == user_id]['num_user_ratings'].iloc[0]
    user_activity = df_users[df_users['user_id'] == user_id]['user_activity'].iloc[0]
    
    # Определяем погоровые значения
    if threshold is None:
        threshold = [3.5, 10, 0.1]

    th_avg_user_rating, th_num_user_ratings, th_user_activity = threshold

    if avg_user_rating >= th_avg_user_rating and \
      (num_user_ratings >= th_num_user_ratings or user_activity >= th_user_activity):
        user_type = 'active'
    else:
        user_type = 'new'

    return user_type

In [30]:
def get_combined_recomendation_by_user_type(user_id, df_ratings, df_books, df_tags, df_book_tags, df_users, book_id=None):
    """Функция комбинирует персонализированные и популярные рекомендаций на основе типа пользователя.
Для новых пользователей показываются популярные книги, активным пользователям предлагаются 
гибридные рекомендации с упором на схожесть интересов.
    :param user_id: идентификатор пользователя
    :param book_id: идентификатор книги
    :param df_ratings: DataFrame с рейтингами
    :param df_books: DataFrame с информацией о книгах
    :param df_tags: DataFrame с тегами
    :param df_book_tags: DataFrame с привязанными тегами к книгам

    :return: список рекомендованных книг
    """
    user_type = get_user_type(user_id, df_users)

    # Новым пользователям предлагаются деперсонализированные популярные книги
    # Активным пользователям - гибридные взвешенные рекомендации
    if user_type == 'new':
        combined_recs = get_popularity_recommendation_ids(df_ratings)
    elif user_type == 'active':
        combined_recs = get_hybrid_recommendation(user_id, df_ratings, df_books, df_tags, df_book_tags, book_id)

    return combined_recs

## Система генерации кандидатов

In [32]:
def get_candidate_pool(user_id, df_ratings, df_books, df_tags, df_book_tags, book_id=None, N=5):
    """Функция объединяет рекомендации от всех моделей в общий пул с учетом уже прочитанных книг.
    :param user_id: идентификатор пользователя
    :param book_id: идентификатор книги
    :param df_ratings: DataFrame с рейтингами
    :param df_books: DataFrame с информацией о книгах
    :param df_tags: DataFrame с тегами
    :param df_book_tags: DataFrame с привязанными тегами к книгам
    
    :return: список уникальных кандидатских книг
    """
    # Получаем полные рекомендации из разных моделей
    weights = [N, N, N]
    candidate_pool = get_hybrid_recommendation(user_id, df_ratings, df_books, df_tags, df_book_tags, book_id, weights=weights)
        
    # Исключаем прочитанные книги
    read_books = set(df_ratings[df_ratings['user_id'] == user_id]['book_id'])
    final_candidates = list(set(candidate_pool) - read_books)
    
    return final_candidates

In [76]:
def set_diversity_filter(candidates, df_books, max_genre_ratio=0.6):
    """Функция проводит балансировку между разнообразием и релевантностью,
путем ограничения доминирования определённого жанра или автора.
    :param candidates: список книг-кандидатов
    :param df_books: DataFrame с информацией о книгах
    :param max_genre_ratio: максимальный процент книг одного жанра или автора
    
    :return: сбалансированный список кандидатов
    """
    filtered_candidates = []
    tag_counts = {}
    author_counts = {}
    
    for book_id in candidates:
        row = df_books[df_books['book_id'] == book_id]
        if len(row) == 0:
            filtered_candidates.append(book_id)
            continue
        row = row.iloc[0]
        tag = row['top_tag']
        author = row['authors']
        
        # Считаем частоту появления топ-тэга и автора
        tag_counts.setdefault(tag, 0)
        author_counts.setdefault(author, 0)
        tag_counts[tag] += 1
        author_counts[author] += 1
        
        # Применяем ограничение по тэгу или автору
        if tag_counts[tag] <= len(candidates)*max_genre_ratio and author_counts[author] <= len(candidates)*max_genre_ratio:
            filtered_candidates.append(book_id)
            
    return filtered_candidates

# Запуск

In [77]:
# Определяем запуск из-под скрипта:
if __name__ == '__main__':
    data_path = os.path.abspath('../data')
    # book_id = 6621
    user_id = 315

    # Загружаем даныне
    print('Загружаем данные...')
    df_ratings, df_books, df_tags, df_book_tags, df_users = make_datasets(data_path)

    # Создаем матрицу взаимодействий
    print('Создаем матрицу взаимодействий...')
    # df_interaction_matrix = generate_user_book_similarity_matrix(df_ratings, df_books, df_book_tags, df_tags)
    
    # Обучаем модель SVD
    print('Обучаем модель SVD...')
    # models.best_params = prepare_model_svd_mode(df_ratings, 'increment')
    models.best_params = {'n_factors': 100, 'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.1}

    # Получаем тип пользователя
    print('Получаем тип пользователя...')
    user_type = get_user_type(user_id, df_users)
    print(user_type)

    # Получаем комбинированные рекомендации в зависимости от типа пользователя
    print('Получаем комбинированные рекомендации в зависимости от типа пользователя...')
    combined_recs = get_combined_recomendation_by_user_type(user_id, df_ratings, df_books, df_tags, df_book_tags, df_users)
    print(combined_recs)

    # Получаем общий пул рекомендаций с фильтрацией по прочитанным книгам
    print('Получаем общий пул рекомендаций с фильтрацией по прочитанным книгам...')
    final_candidates = get_candidate_pool(user_id, df_ratings, df_books, df_tags, df_book_tags)
    print(final_candidates)

    # Получаем сбалансированный список кандидатов
    print('Получаем сбалансированный список кандидатов...')
    filtered_candidates = set_diversity_filter(final_candidates, df_books)
    print(filtered_candidates)

Загружаем данные...
Создаем матрицу взаимодействий...
Обучаем модель SVD...
Получаем тип пользователя...
active
Получаем комбинированные рекомендации в зависимости от типа пользователя...
[2850, 3080, 6920, 4557, 8946, 8854, 5207]
Получаем общий пул рекомендаций с фильтрацией по прочитанным книгам...
[2850, 4098, 3080, 6920, 3628, 4557, 5207, 4058, 1788, 9566]
Получаем сбалансированный список кандидатов...
[2850, 4098, 3080, 6920, 3628, 4557, 5207, 4058, 1788, 9566]
